## DDPM Generate an image using Diffusers 
- 영상 생성하는 reverse process를 살펴보자.
- forward process를 기억하며 이해해 보자.

### 참조
- Reference [diffusers_intro](https://github.com/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb), [training_example](https://github.com/huggingface/notebooks/blob/main/diffusers/training_example.ipynb)
- Requirements: diffusers, pytorch 

### Installation
- Install `!pip install diffusers==0.11.1`
- Install `!pip install accelerate`  
  if you see the following message    
  ```
  Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
    - Install `pip install diffusers[training]==0.11.1
  ```

In [ ]:
from IPython.display import Image as DisplayImage

# disable warning message
import torchvision 
torchvision.disable_beta_transforms_warning()

from diffusers import DDPMPipeline

import torch
import time

In [ ]:
n_drive = '\\\\swschoolavdazfiles002.file.core.windows.net\\aias-vision'
dataset_path = n_drive + '\\' + 'AI-Application-Specialist-Vision-Dataset' 

DisplayImage(filename=dataset_path  + "\\" + 'hf-assets/ddpm_paper.png', width=400)

In [ ]:
import sys  # 'linux', 'win32'
import os #'nt','posix'
if sys.platform == 'linux': # mlp
    # in order to download models from huggingface in the ML Platform, it is necessary to set the following proxy and ssl 
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    # suwon
    import os
    os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'
    os.environ['HTTP_PROXY'] ='http://75.17.107.42:8080'
    os.environ['HTTPS_PROXY'] ='http://75.17.107.42:8080'
elif sys.platform == 'win32':
    os.environ["PATH"]+= os.pathsep+"C:\\Program Files\\Graphviz\\bin"

In [ ]:
import os
cache_path = 'd:/HF_cache'
os.environ['HF_HOME'] = cache_path
os.environ['TRANSFORMERS_CACHE'] = cache_path # seems not to work
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = 'false' # hg download 시 warning disable 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hf_model_dir = dataset_path + "hf-models/"
download_from_local = False # True if downloading from this local system, False if downloading from remote huggingface sites

In [ ]:
model_id = "google/ddpm-cat-256"
# model_id = "google/ddpm-celebahq-256"
# model_id = "google/ddpm-church-256"

if download_from_local == True: # download from local folder 
    repo_id = hf_model_dir+model_id
else: # directly to download from huggingface 
    repo_id = model_id
    
ddpm_pipe = DDPMPipeline.from_pretrained(repo_id, cache_dir=cache_path)
ddpm_pipe.to(device)
ddpm_pipe

## 1. pre-trained model로 image를 생성하는 방법
- diffusion steps로 sample하는 방법

In [ ]:
# num_inference_steps: the number of diffusion steps used when generating samples with a pre-trained model
# ddpm num_inference_steps = 1000
images = ddpm_pipe().images
images[0]

In [ ]:
len(images)

## 2. sampling step을 단계적으로 진행
- huggingface DDPMPipeline은 UNet2DModel과 DDPMScheduler로 구성됨
- 표준 정규 분포에서 noise image sample 생성 ($x_T$)
- UNet2DModel().sample 은 현재 noisy sample($x_t$), t를 입력으로 noise residual($e_t$)을 
  prediction함  
  : $(x_t, t)$ &rarr; $e_t$
- DDPMScheduler.step().prev_sample: noise residual을 사용하여 less noisy image를 sampling함  
  : ($e_t, t, x_t$) &rarr; $x_{t-1}$

In [ ]:
from diffusers import UNet2DModel

model_id = "google/ddpm-church-256"

if download_from_local == True:
    repo_id = hf_model_dir + model_id
else: 
    repo_id = model_id
    
model = UNet2DModel.from_pretrained(repo_id, cache_dir=cache_path)

In [ ]:
model

In [ ]:
model.config

In [ ]:
# random image at time=T
torch.manual_seed(0)

# 1) to sample a random noise image [C, H, W] from a Normal distribution N(0, I)
noisy_sample = torch.randn(
    1, model.config.in_channels, model.config.sample_size, model.config.sample_size
)
noisy_sample.shape

In [ ]:
import PIL.Image
import numpy as np

def display_sample(sample, t):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127 # 127.5
    image_processed = torch.clamp(image_processed, min=0, max=255)
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {t}")
    display(image_pil)

In [ ]:
display_sample(noisy_sample,999)

In [ ]:
# 2) model을 사용하여 noisy_residual 을 prediction함
# - input: sample(현재 noise sample), timestep (t)
# - output: previous noisy residual
with torch.no_grad():
    noisy_residual = model(sample=noisy_sample, timestep=999).sample

In [ ]:
noisy_residual.shape

In [ ]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler.from_pretrained(repo_id)

In [ ]:
scheduler.config

In [ ]:
# 3) model을 이용하여 timestep t에서 예측된 noisy_residual를 사용하여  
# 좀더 noisy가 적은(less_noisy_sample image) t-1의 previsous image를 sampling 
# : DDPM reverse process - sampling 단계
less_noisy_sample = scheduler.step(
    model_output=noisy_residual, timestep=999, sample=noisy_sample
).prev_sample

In [ ]:
noisy_residual.shape, less_noisy_sample.shape

In [ ]:
display_sample(less_noisy_sample,998)

In [ ]:
# from 'cpu' to 'cuda' gpu
model.to(device)
noisy_sample = noisy_sample.to(device)

### 4) model과 scheduler를 사용하여 연속적으로 timesteps reverse process 진행하면서 중간 결과를 살펴보자.
- noise sample image에서 점차로 영상으로 진행해 가는 과정을 살펴보자

In [ ]:
import tqdm

sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute less noisy image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 50 == 0:
        display_sample(sample, i + 1)

### DDIM Scheduler
- scheduler를 DDIM 방식으로 변경하여 실습해 보자.
- 차이가 무엇인가

In [ ]:
from diffusers import DDIMScheduler

scheduler = DDIMScheduler.from_pretrained(repo_id, cache_dir=cache_path)

In [ ]:
scheduler

In [ ]:
scheduler.set_timesteps(num_inference_steps=50)

In [ ]:
import tqdm

sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute previous image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 10 == 0:
        display_sample(sample, i + 1)

### cifar10-ddpm

In [ ]:
# !pip install diffusers
from diffusers import DDPMPipeline, DDIMPipeline, PNDMPipeline

In [ ]:
# load model and scheduler
if download_from_local == True:
    model_id = hf_model_dir + "google/ddpm-cifar10-32"
else: 
    model_id = "google/ddpm-cifar10-32"
    
ddpm = DDPMPipeline.from_pretrained(model_id, cache_dir=cache_path)  # you can replace DDPMPipeline with DDIMPipeline or PNDMPipeline for faster inference
ddpm = ddpm.to(device)

In [ ]:
# run pipeline in inference (sample random noise and denoise)
t0 = time.time()
image = ddpm().images[0]
print(time.time()-t0)

image

In [ ]:
ddim = DDIMPipeline.from_pretrained(model_id, cache_dir=cache_path)  # you can replace DDPMPipeline with DDIMPipeline or PNDMPipeline for faster inference
ddim = ddim.to(device)

In [ ]:
# run pipeline in inference (sample random noise and denoise)
t0 = time.time()
image = ddim().images[0]
print(time.time()-t0)

# to save image
# image.save("ddim_generated_image.png")

In [ ]:
image